In [1]:
import os
from bs4 import BeautifulSoup
from playwright.async_api import async_playwright, TimeoutError as PlaywrightTimeout
import time

In [2]:
#%pip install beautifulsoup4

In [3]:
#%pip install playwright

In [4]:
#!playwright install

In [5]:
SEASONS = list(range(2016, 2023))

In [6]:
SEASONS

[2016, 2017, 2018, 2019, 2020, 2021, 2022]

In [7]:
DATA_DIR = 'data'
STANDINGS_DIR = os.path.join(DATA_DIR, 'standings')
SCORE_DIR = os.path.join(DATA_DIR, 'scores')

In [8]:
async def get_html(url, selector, sleep=5, retries=3):
    html = None
    for i in range(1, retries+1):
        time.sleep(sleep * i)
        try:
            async with async_playwright() as p:
                browser = await p.chromium.launch()
                page = await browser.new_page()
                await page.goto(url)
                print(await page.title())
                html = await page.inner_html(selector)
        except PlaywrightTimeout:
            print(f"Timeout error on {url}")
            continue
        else:
            break
    return html

In [9]:
# season = 2016

# url = f"https://www.basketball-reference.com/leagues/NBA_{season}_games.html"

In [10]:
# url

In [11]:
async def scrape_season(season):
    url = f"https://www.basketball-reference.com/leagues/NBA_{season}_games.html"
    html = await get_html(url, "#content .filter")
    
    soup = BeautifulSoup(html)
    links = soup.find_all("a")
    standings_pages = [f"https://www.basketball-reference.com{l['href']}" for l in links]
    
    for url in standings_pages:
        save_path = os.path.join(STANDINGS_DIR, url.split("/")[-1])
        if os.path.exists(save_path):
            continue
        
        html = await get_html(url, "#all_schedule")
        with open(save_path, "w+") as f:
            f.write(html)

In [12]:
pip install -U jupyter

Requirement already up-to-date: jupyter in c:\users\syahru\anaconda3\lib\site-packages (1.0.0)



In [13]:
for season in SEASONS:
    await scrape_season(season)

Task exception was never retrieved
future: <Task finished name='Task-2' coro=<Connection.run() done, defined at C:\Users\syahru\anaconda3\lib\site-packages\playwright\_impl\_connection.py:217> exception=NotImplementedError()>
Traceback (most recent call last):
  File "C:\Users\syahru\anaconda3\lib\site-packages\playwright\_impl\_connection.py", line 224, in run
    await self._transport.connect()
  File "C:\Users\syahru\anaconda3\lib\site-packages\playwright\_impl\_transport.py", line 133, in connect
    raise exc
  File "C:\Users\syahru\anaconda3\lib\site-packages\playwright\_impl\_transport.py", line 121, in connect
    self._proc = await asyncio.create_subprocess_exec(
  File "C:\Users\syahru\anaconda3\lib\asyncio\subprocess.py", line 236, in create_subprocess_exec
    transport, protocol = await loop.subprocess_exec(
  File "C:\Users\syahru\anaconda3\lib\asyncio\base_events.py", line 1630, in subprocess_exec
    transport = await self._make_subprocess_transport(
  File "C:\Users\sy

NotImplementedError: 

In [ ]:
standings_files = os.listdir(STANDINGS_DIR)

In [ ]:
async def scrape_game(standings_file):
    with open(standings_file, 'r') as f:
        html = f.read()

    soup = BeautifulSoup(html)
    links = soup.find_all("a")
    hrefs = [l.get('href') for l in links]
    box_scores = [f"https://www.basketball-reference.com{l}" for l in hrefs if l and "boxscore" in l and '.html' in l]

    for url in box_scores:
        save_path = os.path.join(SCORES_DIR, url.split("/")[-1])
        if os.path.exists(save_path):
            continue

        html = await get_html(url, "#content")
        if not html:
            continue
        with open(save_path, "w+") as f:
            f.write(html)

In [ ]:
import pandas as pd

for season in SEASONS:
    files = [s for s in standings_files if str(season) in s]
    
    for f in files:
        filepath = os.path.join(STANDINGS_DIR, f)
        
        await scrape_game(filepath)

In [ ]:
# html = await get_html(url, "#content, filter")